In [121]:
import os
import gc
import random
import pandas as pd
import numpy as np
import lightgbm as lgb

from tqdm.notebook import tqdm
from tqdm import tqdm_notebook

from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.simplefilter('ignore')
class Process :

  def LAG(self,data,LagFeature,shift=1,NewFeatures=[]) :
    data[NewFeatures[0]]   = data[LagFeature]  - data[LagFeature].shift(shift)
    data[NewFeatures[1]]   = data[LagFeature].shift(shift)

  def LE(self,data,LE_cols = []) :
    LE = LabelEncoder()
    for le_col in LE_cols :
      data[le_col] = LE.fit_transform(data[le_col])
  
  def process(self,train,test) :
    data = pd.concat([train,test]).reset_index(drop=True)
    from sklearn.experimental import enable_iterative_imputer 
    from sklearn.impute import IterativeImputer
    from sklearn import preprocessing
    le = preprocessing.LabelEncoder()
    le.fit(data['device'])
    date=pd.DataFrame(data["date"])
    l=data["ID"]
    data['device']=le.transform(data['device'])
    data=data.drop(["ID","date"], axis=1)

    mice = IterativeImputer(random_state=42, verbose=1)
    data=pd.DataFrame(mice.fit_transform(data),columns=data.columns)
    
    # time features 
    new = date["date"].str.split("-", n = 2, expand = True)
    data['year']=new[0].astype(int)
    data['month']=new[1].astype(int)
    data['day']=new[2].astype(int)

#     data['created_at'] = pd.to_datetime(date)
#     data['year'] = data['created_at'].dt.year
#     data['year'] = data['year'].astype(float)
#     data['month'] = data['created_at'].dt.month
#     data['day'] = data['created_at'].dt.day
    #######################

    #######################
    # combination between time features
    data['month_day'] = data['month'].astype(str) + '-' + data['day'].astype(str)

    # Label Encoder
    self.LE(data,LE_cols = ['month_day'])

    # FE
    data = data.sort_values('month_day').reset_index(drop=True)
    self.LAG(data,LagFeature='SulphurDioxide_SO2_column_number_density',shift=1,NewFeatures=['sensor1_diff_next','sensor1_next'])
    self.LAG(data,LagFeature='SulphurDioxide_SO2_column_number_density',shift=-1,NewFeatures=['sensor1_diff_before','sensor1_before'])
    self.LAG(data,LagFeature='SulphurDioxide_SO2_column_number_density',shift=1,NewFeatures=['sensor2_diff_next','sensor2_next'])
    self.LAG(data,LagFeature='SulphurDioxide_SO2_column_number_density',shift=-1,NewFeatures=['sensor2_diff_before','sensor2_before'])

    # Get our New Train,Test
    data['SplitBy']   = data['year'].astype(int).astype(str) + '-' + data['month'].astype(str) + '-' + data['day'].astype(str)
    data = data.sort_values('SplitBy').reset_index(drop=True)
    train = data[l.isin(train['ID'].values)].reset_index(drop=True)

    test = data[~l.isin(train['ID'].values)].reset_index(drop=True)
    #####################


    #####################
#     from sklearn.preprocessing import MinMaxScaler, LabelEncoder
#     minmax = MinMaxScaler()
#     train=pd.DataFrame(minmax.fit_transform(train), columns=train.columns)
#     test=pd.DataFrame(minmax.transform(X_test), columns=test.columns)
    #######################
    #######################

    

    return train, test


In [122]:
train = pd.read_csv('../input/layerai-air-quality-prediction-challenge/test (14).csv')
test = pd.read_csv('../input/layerai-air-quality-prediction-challenge/train (13).csv')

In [123]:
l=train["ID"]

In [124]:
train.head()

,ID,date,device,site_latitude,site_longitude,humidity,temp_mean,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,...,Cloud_cloud_top_pressure,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle
0,ID_UOH62J0XHX,2020-10-02,aq_43,0.289040,32.589580,0.756840,23.849653,0.000124,0.839054,0.000104,...,36716.141728,8325.718024,41969.524699,7325.718482,2.632129,0.210430,74.431975,35.449770,-101.546713,21.006060
1,ID_4OPWSB0UHJ,2020-12-07,aq_54,0.356400,32.573000,0.810854,22.075347,NaN,NaN,NaN,...,63086.023438,4060.879639,70871.984375,3060.879639,2.773685,0.344436,-97.300392,63.489811,-125.937134,41.709042
2,ID_3SLMNNG1Z3,2020-11-16,aq_59,0.381576,32.647109,0.883253,20.050694,NaN,NaN,NaN,...,46783.820820,6456.160528,53084.224103,5456.160528,3.862119,0.326882,-97.699519,58.824120,-120.683819,40.183680
3,ID_MFUHTXYPM4,2020-09-05,AW66FF7V,0.379562,32.553714,0.782583,22.248958,0.000176,0.730853,0.000129,...,45528.261719,6682.570801,51503.203125,5682.570801,1.937542,0.212181,75.361595,24.835907,-72.224319,20.675266
4,ID_TY1DAND8ZP,2020-12-15,aq_45,0.344000,32.553000,0.770116,22.565217,NaN,NaN,NaN,...,65512.797785,3757.637022,73461.827324,2757.637022,3.733908,0.262021,76.537080,11.026513,-139.072620,32.046360


In [125]:
train

,ID,date,device,site_latitude,site_longitude,humidity,temp_mean,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,...,Cloud_cloud_top_pressure,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle
0,ID_UOH62J0XHX,2020-10-02,aq_43,0.289040,32.589580,0.756840,23.849653,0.000124,0.839054,0.000104,...,36716.141728,8325.718024,41969.524699,7325.718482,2.632129,0.210430,74.431975,35.449770,-101.546713,21.006060
1,ID_4OPWSB0UHJ,2020-12-07,aq_54,0.356400,32.573000,0.810854,22.075347,NaN,NaN,NaN,...,63086.023438,4060.879639,70871.984375,3060.879639,2.773685,0.344436,-97.300392,63.489811,-125.937134,41.709042
2,ID_3SLMNNG1Z3,2020-11-16,aq_59,0.381576,32.647109,0.883253,20.050694,NaN,NaN,NaN,...,46783.820820,6456.160528,53084.224103,5456.160528,3.862119,0.326882,-97.699519,58.824120,-120.683819,40.183680
3,ID_MFUHTXYPM4,2020-09-05,AW66FF7V,0.379562,32.553714,0.782583,22.248958,0.000176,0.730853,0.000129,...,45528.261719,6682.570801,51503.203125,5682.570801,1.937542,0.212181,75.361595,24.835907,-72.224319,20.675266
4,ID_TY1DAND8ZP,2020-12-15,aq_45,0.344000,32.553000,0.770116,22.565217,NaN,NaN,NaN,...,65512.797785,3757.637022,73461.827324,2757.637022,3.733908,0.262021,76.537080,11.026513,-139.072620,32.046360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249,ID_NYKGOYFBKZ,2020-08-25,aq_76,0.357987,32.584100,0.786575,23.790833,NaN,NaN,NaN,...,52109.843750,5625.392090,59100.105469,4625.392090,11.611624,0.235978,76.397339,12.520192,-62.776257,22.760517
4250,ID_IWO4EVLHSQ,2020-08-21,AB6051M4,0.390741,32.582257,0.828441,21.801389,NaN,NaN,NaN,...,29895.648437,9842.744141,34098.406250,8842.744141,7.293560,0.285149,72.767441,51.517994,-49.859035,18.032558
4251,ID_G1M1IMY9WU,2020-11-07,AB6051M4,0.390741,32.582257,0.859490,21.851042,NaN,NaN,NaN,...,31481.298828,9398.727539,36332.582031,8398.727539,4.397998,0.292733,-101.590965,15.151692,-122.484474,32.596294
4252,ID_DXV65Q1QXI,2020-09-10,aq_61,0.373078,32.628226,0.833803,21.948864,NaN,NaN,NaN,...,82490.293714,1776.181599,85711.453792,1441.561424,5.902633,0.216981,76.433538,12.068682,-78.491224,22.234252


In [126]:

processor = Process()
train, test= processor.process(train,test)


[IterativeImputer] Completing matrix with shape (14177, 69)
[IterativeImputer] Change: 299007.30617367226, scaled tolerance: 830.2385689972649 
[IterativeImputer] Change: 3038454.3589014797, scaled tolerance: 830.2385689972649 
[IterativeImputer] Change: 4221001.4996927045, scaled tolerance: 830.2385689972649 
[IterativeImputer] Change: 4193379.558430845, scaled tolerance: 830.2385689972649 
[IterativeImputer] Change: 3679422.4111437527, scaled tolerance: 830.2385689972649 
[IterativeImputer] Change: 1882505.362581613, scaled tolerance: 830.2385689972649 
[IterativeImputer] Change: 772540.996334518, scaled tolerance: 830.2385689972649 
[IterativeImputer] Change: 599584.3134162829, scaled tolerance: 830.2385689972649 
[IterativeImputer] Change: 878153.4989488345, scaled tolerance: 830.2385689972649 
[IterativeImputer] Change: 622940.3021738186, scaled tolerance: 830.2385689972649 


KeyError: 'ID'

In [ ]:
print('unique days',train.SplitBy.nunique() , test.SplitBy.nunique())
print('shape',train.shape , test.shape)

In [ ]:
class CFG :
  SEED = 42
  n_splits = 5

  lgb_params = {'boosting_type': 'gbdt','objective': 'binary','metric': 'auc',
                'n_estimators': 2500,'reg_lambda' :50,'num_leaves' : 81,
                'seed': SEED,'silent':True,'early_stopping_rounds': 100,
               }
  remove_features = ['ID',"created_at",'SplitBy' , 'folds', 'pm2_5',]
  TARGET_COL = 'pm2_5'
class CostumSplit :
  def __init__(self,) :
    self.n_splits = CFG.n_splits

  def Split(self,Train_) :
    kf = GroupKFold(n_splits=self.n_splits)

    Train = Train_.copy()
    Train = Train.drop_duplicates('SplitBy').reset_index(drop=True)
    
    groups = Train['SplitBy']
    Train["folds"]=-1   
    for fold, (_, val_index) in enumerate(kf.split(Train,Train['Offset_fault'],groups)):
          Train.loc[val_index, "folds"] = fold
    return Train

  def apply(self,train) :
    mapper = dict(zip(self.Split(train)['SplitBy'].tolist(),
                      self.Split(train)['folds'].tolist()))

    train['folds'] = train['SplitBy'].map(mapper)
    return train

In [ ]:
split = CostumSplit() 

train = split.apply(train)
features_columns = [col for col in train.columns if col not in CFG.remove_features]
len(features_columns)

In [ ]:
oof_lgb = np.zeros((train.shape[0],))
test[CFG.TARGET_COL] = 0
lgb_preds = []

for fold in range(CFG.n_splits) :
    print(50*'-')
    print(f'Fold {fold+1} / {CFG.n_splits}' )

    tr_x, tr_y = train[train['folds']!=fold][features_columns] , train[train['folds']!=fold][CFG.TARGET_COL] 
    vl_x, vl_y = train[train['folds']==fold][features_columns] , train[train['folds']==fold][CFG.TARGET_COL] 
    val_idx = vl_x.index.tolist()

    train_data = lgb.Dataset(tr_x, label=tr_y)
    valid_data = lgb.Dataset(vl_x, label=vl_y)

    estimator = lgb.train(CFG.lgb_params,train_data,valid_sets = [train_data,valid_data],verbose_eval = 0)
    
    y_pred_val = estimator.predict(vl_x,num_iteration=estimator.best_iteration)
    oof_lgb[val_idx] = y_pred_val
    print(f'FOLD-{fold} accuracy score :',accuracy_score(vl_y, (y_pred_val>0.5)*1))

    y_pred_test = estimator.predict(test[features_columns],num_iteration=estimator.best_iteration)
    lgb_preds.append(y_pred_test)
    print(50*'-')

print('OOF score :',accuracy_score(train[CFG.TARGET_COL], (oof_lgb>0.5)*1))